<a href="https://colab.research.google.com/github/MustofAhmed41/Exercise-Pattern-Prediction-Deep-Learning/blob/master/CNN%20%2B%20LSTM%20with%20padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Library

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from tensorflow.keras.utils import to_categorical
from numpy import mean
from numpy import std
from numpy import dstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D

# Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd'/content/drive/MyDrive/Colab Notebooks/Dataset/Softcom Assignment 1'

/content/drive/MyDrive/Colab Notebooks/Dataset/Softcom Assignment 1


# Loading Data

In [ ]:
load = pd.read_csv('Preprocessed_Exercise_Deep_Learning.csv');

In [ ]:
df = load.copy()

In [ ]:
df.head()

,Unnamed: 0,new_window,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,kurtosis_roll_belt,kurtosis_picth_belt,kurtosis_yaw_belt,...,gyros_dumbbell_x,gyros_dumbbell_y,gyros_dumbbell_z,accel_dumbbell_x,accel_dumbbell_y,accel_dumbbell_z,magnet_dumbbell_x,magnet_dumbbell_y,magnet_dumbbell_z,classe
0,0,no,1.0,3.70,41.6,-82.8,3.0,-0.157538,-1.610293,-0.157538,...,-0.31,0.16,0.08,5.0,21.0,37.0,-471.0,191.0,277.0,E
1,1,no,1.0,3.66,42.8,-82.5,2.0,-0.157538,-1.610293,-0.157538,...,-0.31,0.14,0.07,4.0,22.0,35.0,-472.0,184.0,281.0,E
2,2,no,1.0,3.58,43.7,-82.3,1.0,-0.157538,-1.610293,-0.157538,...,-0.31,0.16,0.05,3.0,23.0,37.0,-468.0,190.0,275.0,E
3,3,no,1.0,3.56,44.4,-82.1,1.0,-0.157538,-1.610293,-0.157538,...,-0.31,0.16,0.07,5.0,24.0,38.0,-469.0,184.0,285.0,E
4,4,no,1.0,3.57,45.1,-81.9,1.0,-0.157538,-1.610293,-0.157538,...,-0.31,0.14,0.07,5.0,23.0,37.0,-468.0,189.0,292.0,E


In [ ]:
#converting dataframe to numpy arrays

df2 = pd.DataFrame(data = np.array(df['num_window']), columns=['num_window'] )
df2['roll_belt'], df2['pitch_belt'], df2['yaw_belt'] = np.array(df['roll_belt']),  np.array(df['pitch_belt']), np.array(df['yaw_belt'])
df2['roll_dumbbell'], df2['pitch_dumbbell'], df2['yaw_dumbbell']  = np.array(df['roll_dumbbell']),  np.array(df['pitch_dumbbell']), np.array(df['yaw_dumbbell'])
df2['total_accel_belt'], df2['total_accel_dumbbell']  = np.array(df['total_accel_belt']),  np.array(df['total_accel_dumbbell'])
df2['gyros_belt_x'], df2['gyros_belt_y'], df2['gyros_belt_z']  = np.array(df['gyros_belt_x']),  np.array(df['gyros_belt_y']), np.array(df['gyros_belt_z'])
df2['accel_belt_x'], df2['accel_belt_y'], df2['accel_belt_z']  = np.array(df['accel_belt_x']),  np.array(df['accel_belt_y']), np.array(df['accel_belt_z'])
df2['magnet_belt_x'], df2['magnet_belt_y'], df2['magnet_belt_z']  = np.array(df['magnet_belt_x']),  np.array(df['magnet_belt_y']), np.array(df['magnet_belt_z'])
df2['gyros_dumbbell_x'], df2['gyros_dumbbell_y'], df2['gyros_dumbbell_z']  = np.array(df['gyros_dumbbell_x']),  np.array(df['gyros_dumbbell_y']), np.array(df['gyros_dumbbell_z'])
df2['accel_dumbbell_x'], df2['accel_dumbbell_y'], df2['accel_dumbbell_z']  = np.array(df['accel_dumbbell_x']),  np.array(df['accel_dumbbell_y']), np.array(df['accel_dumbbell_z'])
df2['magnet_dumbbell_x'], df2['magnet_dumbbell_y'], df2['magnet_dumbbell_z']  = np.array(df['magnet_dumbbell_x']),  np.array(df['magnet_dumbbell_y']), np.array(df['magnet_dumbbell_z'])
df2['classe'] =  np.array(df['classe'])

In [ ]:
df2.columns

Index(['num_window', 'roll_belt', 'pitch_belt', 'yaw_belt', 'roll_dumbbell',
       'pitch_dumbbell', 'yaw_dumbbell', 'total_accel_belt',
       'total_accel_dumbbell', 'gyros_belt_x', 'gyros_belt_y', 'gyros_belt_z',
       'accel_belt_x', 'accel_belt_y', 'accel_belt_z', 'magnet_belt_x',
       'magnet_belt_y', 'magnet_belt_z', 'gyros_dumbbell_x',
       'gyros_dumbbell_y', 'gyros_dumbbell_z', 'accel_dumbbell_x',
       'accel_dumbbell_y', 'accel_dumbbell_z', 'magnet_dumbbell_x',
       'magnet_dumbbell_y', 'magnet_dumbbell_z', 'classe'],
      dtype='object')

In [ ]:
# Creating data of fixed window size
total = 0
count = 0
dropping_threshold = 27
for i in df2['num_window'].unique():
    val = len( df2[ df2['num_window'] == i ])
    total += val
    if val <=dropping_threshold:
        count += 1
print('Total : ', df2['num_window'].nunique())
print('Average : ', total/df2['num_window'].nunique())
print('Dropped : ', count)

Total :  855
Average :  45.88187134502924
Dropped :  31


In [ ]:
lst = []
for i in df2['num_window'].unique():  
    lst.append(len( df2[ df2['num_window'] == i ]))
max_window = max(lst)
max_window

73

In [ ]:
df2.columns

Index(['num_window', 'roll_belt', 'pitch_belt', 'yaw_belt', 'roll_dumbbell',
       'pitch_dumbbell', 'yaw_dumbbell', 'total_accel_belt',
       'total_accel_dumbbell', 'gyros_belt_x', 'gyros_belt_y', 'gyros_belt_z',
       'accel_belt_x', 'accel_belt_y', 'accel_belt_z', 'magnet_belt_x',
       'magnet_belt_y', 'magnet_belt_z', 'gyros_dumbbell_x',
       'gyros_dumbbell_y', 'gyros_dumbbell_z', 'accel_dumbbell_x',
       'accel_dumbbell_y', 'accel_dumbbell_z', 'magnet_dumbbell_x',
       'magnet_dumbbell_y', 'magnet_dumbbell_z', 'classe'],
      dtype='object')

In [ ]:
print(len(df2))

39229


In [ ]:
# PADDING

for i in df2['num_window'].unique():
    temp = df2[df2['num_window'] == i]
    label = df2[df2['num_window'] == i]['classe'].iloc[0]
    if(len(temp)<max_window):
        rows_to_add = max_window - len(temp)
        new = pd.DataFrame( np.zeros((rows_to_add, len(df2.columns))) , columns=df2.columns)
        new['num_window'] = i
        new['classe'] = label
        df2 = df2.append(new).reset_index(drop=True)

In [ ]:
print(len(df2))

62415


In [ ]:
def impute_classe(x):
  if x == 'A':
    return 5
  elif x =='B':
    return 4
  elif x=='C':
    return 3
  elif x=='D':
    return 2
  elif x=='E':
    return 1

In [ ]:
df2['classe'] = df2['classe'].apply(impute_classe)

In [ ]:
column_names = df2.drop(['classe','num_window'], axis=1).columns

In [ ]:
# Preparing data for CNN

global v_stacked
global v_stacked_2
window_size = dropping_threshold
counter = 0
v_stacked_2 = np.zeros((1,1))
target_label = []
for i in df2['num_window'].unique():
    temp = df2[df2['num_window'] == i]
    label = df2[df2['num_window'] == i]['classe'].iloc[0]    
    v_stacked = np.zeros(shape=(1, 1))
    for k in column_names:
        temp2 = df2[df2['num_window'] == i][k].to_numpy()
        #temp2 = temp2.reshape(-1,1)          
        if v_stacked.shape[0] == 1 and v_stacked.shape[1] == 1:
            v_stacked = temp2 
        else:
            v_stacked = np.dstack((v_stacked, temp2))   
    target_label.append(label)        
    if v_stacked_2.shape[0] == 1 and v_stacked_2.shape[1] == 1:
        v_stacked_2 = v_stacked
    else:
        v_stacked_2 = np.vstack((v_stacked_2, v_stacked))

In [ ]:
print(v_stacked_2.shape)

(855, 73, 26)


In [ ]:
print(v_stacked_2[0].shape)

(73, 26)


In [ ]:
print(len(target_label))

855


In [ ]:
target_label = to_categorical(target_label)

In [ ]:
trainX = v_stacked_2[0:780][:][:]
print(trainX.shape)
testX = v_stacked_2[780:-1][:][:]
print(testX.shape)
trainy = target_label[0:780][:][:]
print(trainy.shape)
testy = target_label[780:-1][:][:]
print(testy.shape)

(780, 73, 26)
(74, 73, 26)
(780, 6)
(74, 6)


In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 80, 8
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(Conv1D(filters=6, kernel_size=3, activation='relu', padding='same',input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=9, kernel_size=3, activation='relu', padding='same'))
	model.add(Dropout(0.25))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Conv1D(filters=12, kernel_size=3, activation='relu', padding='same'))
	model.add(LSTM(12, activation='relu'))
	model.add(Dropout(0.15))
	model.add(Flatten())
	model.add(Dense(400, activation='relu'))
	model.add(Dropout(0.10))
	model.add(Dense(700, activation='relu'))
	model.add(Dropout(0.15))
	model.add(Dense(1000, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [ ]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
def run_experiment(repeats=10):
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [ ]:
run_experiment()

>#1: 28.378
>#2: 17.568
>#3: 24.324
>#4: 22.973
>#5: 24.324
>#6: 21.622
>#7: 41.892
>#8: 33.784
>#9: 40.541
>#10: 39.189
[28.378379344940186, 17.56756752729416, 24.32432472705841, 22.972972691059113, 24.32432472705841, 21.621622145175934, 41.89189076423645, 33.78378450870514, 40.54054021835327, 39.18918967247009]
Accuracy: 29.459% (+/-8.304)
